In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time


In [49]:
def get_restaurant_urls(driver):
    
    restaurant_urls = []
    restaurant_elements = driver.find_elements_by_class_name('restaurant.js-restaurant')
    for single_restaurant_element in restaurant_elements:
        try:
            url_element = single_restaurant_element.find_element_by_class_name("restaurantname.notranslate")
            restaurant_urls.append(url_element.get_attribute("href"))
        except:
            print("Error in retrieving one URL: Skip URL")
    return restaurant_urls
    
def start_webdriver(start_url):
    driver = webdriver.Safari()
    time.sleep(0.5)
    driver.get(start_url)
    time.sleep(5)
    
    return driver
    

def get_restaurant_name(driver):
    try:
        description = driver.find_element_by_name('description').get_attribute("content").split(".")[0]
        address = ", ".join(description.split(", ")[1:])
        name = description.split(", ")[0]
    except:
        name = ""
        address = ""
    
    return name, address

def get_food_description(single_food_item):
    try:
        description = single_food_item.find_element_by_class_name("meal__description-additional-info").text
    except:
        description = ""
    return description

def get_food_name(single_food_item):
    try:
        food_class_name = single_food_item.find_element_by_class_name("meal-name")
        food_name_span = food_class_name.find_element_by_class_name("notranslate")
        food_name = food_name_span.get_attribute("data-product-name")
    except:
        food_name = ""
    return food_name

def get_food_price(single_food_item):
    try:
        price = single_food_item.find_element_by_class_name("meal__price.notranslate").text
    except:
        price = ""
    return price

def get_food_category(single_catogory):
    category_name = single_catogory.find_element_by_class_name("menucard__category-name").text
    return category_name

def fill_out_dictionary(data_dictionary, restaurant_name, 
                      restaurant_address, category_name, 
                      all_food_names, all_food_description, 
                      all_food_price):
    
    data_dictionary["restaurant_name"] = restaurant_name
    data_dictionary["location"] = restaurant_address
    data_dictionary["dishes"]["food_name"] = all_food_names
    data_dictionary["dishes"]["price"] = all_food_price
    data_dictionary["dishes"]["category"] = category_name
    data_dictionary["dishes"]["description"] = all_food_description
    
    return data_dictionary
    
    
def get_data(driver):
    
    data_dictionary = {"restaurant_name": "",
                       "location": "",
                       "dishes": {
                           "food_name": "",
                           "price": "",
                           "category": "",
                           "description": ""
                       }}
    
    all_food_names = []
    all_food_price = []
    all_food_description = []
    restaurant_name, restaurant_address = get_restaurant_name(driver)
    menu_split_categories = driver.find_elements_by_class_name("menucard__meals-group")
    #print(restaurant_name)
    for single_catogory in menu_split_categories:
        category_name = get_food_category(single_catogory)
        food_items = single_catogory.find_elements_by_class_name('js-meal-item')

        for single_food_item in food_items:
            all_food_description.append(get_food_description(single_food_item))
            all_food_names.append(get_food_name(single_food_item))
            all_food_price.append(get_food_price(single_food_item))
            
    data_dictionary = fill_out_dictionary(data_dictionary, restaurant_name, 
                                          restaurant_address, category_name, 
                                          all_food_names, all_food_description, 
                                          all_food_price)
    #print("Done with the restaurant.\n")
    return data_dictionary
    

In [50]:
import json
from tqdm import tqdm

start_url = "https://www.lieferando.de/lieferservice/essen/karlsruhe-mannheim-68161"

def scrape_lieferando_pipeline(start_url, batch=10):
    
    json_name = "scraped_data/data_batch_"
    
    driver = start_webdriver(start_url)
    restaurant_urls = get_restaurant_urls(driver)

    restaurant_index = 1
    all_data = []
    for restaurant_link in tqdm(restaurant_urls):
        driver.get(restaurant_link)
        time.sleep(1)
        all_data.append(get_data(driver))
        time.sleep(3)
        if restaurant_index%batch == 0:
            with open(json_name + str(restaurant_index) + ".json", 'w') as f:
                print("Saving Batch")
                json.dump(all_data, f)
        restaurant_index += 1
        
    with open(json_name + "all" + ".json", 'w') as f:
        print("Saving Batch")
        json.dump(all_data, f)
    return all_data
            

In [51]:
start_url = "https://www.lieferando.de/lieferservice/essen/karlsruhe-mannheim-68161"
scraped_data = scrape_lieferando_pipeline(start_url, 50)

Error in retrieving one URL: Skip URL
Saving Batch


Exception ignored in: <function tqdm.__del__ at 0x7f86c320cdc0>
Traceback (most recent call last):
  File "/Users/I534344/opt/anaconda3/envs/ocr_model/lib/python3.8/site-packages/tqdm/std.py", line 1152, in __del__
    self.close()
  File "/Users/I534344/opt/anaconda3/envs/ocr_model/lib/python3.8/site-packages/tqdm/notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x7f86c320cdc0>
Traceback (most recent call last):
  File "/Users/I534344/opt/anaconda3/envs/ocr_model/lib/python3.8/site-packages/tqdm/std.py", line 1152, in __del__
    self.close()
  File "/Users/I534344/opt/anaconda3/envs/ocr_model/lib/python3.8/site-packages/tqdm/notebook.py", line 283, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x7f86c320cdc0>
Trac

Saving Batch
Saving Batch
Saving Batch


In [75]:
list_of_all_food_names = []
for restaurant in scraped_data:
    list_of_all_food_names += restaurant["dishes"]["food_name"]

In [76]:
list_of_all_food_names = list(set(list_of_all_food_names))


In [112]:
for name in lookup_table:
    if "pizza" == name.lower():
        print(name)

pizza
